In [76]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain_community.llms import CTransformers
from llama_cpp import Llama
import os
from dotenv import load_dotenv


In [52]:
load_dotenv()

True

In [53]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_API_ENV = os.getenv("PINECONE_API_ENV")

In [54]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [55]:
BASE_PATH = os.getenv("BASE_PATH")

In [56]:
extracted_data = load_pdf(BASE_PATH+"/AI_Chatbot/Data")

In [57]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [58]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7020


In [59]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [60]:
embeddings = download_hugging_face_embeddings()

In [61]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [62]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [63]:
index_name="chatbot"

In [51]:
#Creating Embeddings for Each of The Text Chunks & storing
#DO NOT RUN THIS STEP for the same document
#docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

docsearch = PineconeVectorStore.from_documents(text_chunks, embeddings, index_name=index_name)

In [64]:
#If we already have an index we can load it like this
docsearch=PineconeVectorStore.from_existing_index(index_name, embeddings)

query = "What are Allergies"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE", metadata={'page': 130.0, 'source': '/home/sbhardwa/AI_Chatbot/Data/Medical_book.pdf'}), Document(page_content='allergens are the following:\n• plant pollens\n• animal fur and dander\n• body parts from house mites (microscopic creatures\nfound in all houses)\n• house dust• mold spores• cigarette smoke• solvents• cleaners\nCommon food allergens include the following:\n• nuts, especially peanuts, walnuts, and brazil nuts\n• fish, mollusks, and shellfish• eggs• 

In [65]:
docsearch

In [66]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [67]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [82]:
from accelerate import Accelerator
accelerator = Accelerator()

In [83]:
config={
    'max_new_tokens':512,
     'temperature':0.8,
     'gpu_layers':50
      }

In [84]:

llm=CTransformers(model="/home/sbhardwa/AI_Chatbot/model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config=config)

In [85]:
llm, config = accelerator.prepare(llm, config)

In [86]:
llm

CTransformers(client=<ctransformers.llm.LLM object at 0x7f706d51e1a0>, model='/home/sbhardwa/AI_Chatbot/model/llama-2-7b-chat.ggmlv3.q4_0.bin', model_type='llama', config={'max_new_tokens': 512, 'temperature': 0.8, 'gpu_layers': 50})

In [87]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [88]:
user_input=input(f"Input Prompt:")
result=qa({"query": user_input})
print("Response : ", result["result"])

Response :  Antibiotics are often prescribed to treat bronchitis, as bacterial infections can be serious if left untreated. However, antibiotics may not be effective against viral infections, which are more common causes of bronchitis. In addition to antibiotics, there are several other treatment options for bronchitis, including:
1. Over-the-counter medications such as cough suppressants and expectorants can help relieve symptoms.
2. Home remedies such as inhaling steam from a bowl of hot water or using a humidifier can help loosen mucus and ease congestion.
3. Drinking plenty of fluids, getting enough rest, and avoiding irritants such as smoke and pollution can also help manage symptoms.
4. In severe cases of bronchitis, hospitalization, hospitalization, hospitalization, hospitalization, hospitalization may require hospitalization, hospitalization, hospitalization, hospitalization, hospitalization, hospitalization, hospitalization, hospitalization, hospitalization, hospitalization, h